In [2]:
import networkx as nx

# 1. Creating a custom graph

In [55]:
# Create an empty undirected graph
G = nx.Graph()

In [56]:
C1 = [(0, 1), (1, 2), (2, 3)]
C2 = [(4, 5), (5, 6)]

G.add_edges_from(C1)
G.add_edges_from(C2)
# G.add_edges_from([
#     (1, 2),
#     (2, 3),
#     (3, 4)
# ])

# G.add_edges_from([
#     (5, 6),
#     (6, 7),
# ])

In [57]:
# Verifying components
list(nx.connected_components(G))

[{0, 1, 2, 3}, {4, 5, 6}]

In [58]:
G.nodes

NodeView((0, 1, 2, 3, 4, 5, 6))

In [59]:
len(G.nodes)

7

In [60]:
G.edges

EdgeView([(0, 1), (1, 2), (2, 3), (4, 5), (5, 6)])

In [61]:
len(G.edges)

5

## BFS + Connected components algorithm for PC computing

In [88]:
from collections import deque

def bfs(G: nx.Graph, source: int, visited: list[bool]) -> list:
  queue = deque()
  res = []

  visited[source] = True
  queue.append(source)

  while len(queue) > 0:

    v = queue.pop()
    res.append(v)
    # print(f"queue: {v}")

    for u in G.neighbors(v):
      if not visited[u]:
        visited[u] = True
        queue.append(u)
        # print(f"queue append: {u}")

  return res

In [95]:
def connected_components(G: nx.Graph):
  visited = [False] * len(G.nodes)
  components = []

  for v in list(G.nodes):
    if not visited[v]:
      comp = bfs(G, v, visited)
      components.append(comp)

  return components

In [94]:
components = connected_components(G)
components

[False, False, False, False, False, False, False]
[True, True, True, True, False, False, False]


[[0, 1, 2, 3], [4, 5, 6]]

- Considering terminal nodes

In [96]:
def create_custom_graph_with_2_comps(terminal_nodes: list[int]) -> nx.Graph:

  G = nx.Graph()

  C1 = [(0, 1), (1, 2), (2, 3)]
  C2 = [(4, 5), (5, 6)]

  G.add_edges_from(C1)
  G.add_edges_from(C2)

  # Add terminal attribute to nodes
  for v in G.nodes:
    if v in terminal_nodes:
      G.nodes[v]["terminal"] = True
    else:
      G.nodes[v]["terminal"] = False
  
  return G

In [111]:
TERMINAL_NODES = [0, 1, 3, 5, 6]

G = create_custom_graph_with_2_comps(terminal_nodes=TERMINAL_NODES)

In [112]:
G.nodes

NodeView((0, 1, 2, 3, 4, 5, 6))

In [113]:
G.nodes(data="terminal")

NodeDataView({0: True, 1: True, 2: False, 3: True, 4: False, 5: True, 6: True}, data='terminal')

In [114]:
def exclude_non_terminal_nodes_from_component(terminal_nodes: list[int], components: list[list[int]]) -> list[list[int]]:

  excluded_components = []

  for comp in components:
    excluded_comp = []
    for v in comp:
      if v in terminal_nodes:
        excluded_comp.append(v)
      
    excluded_components.append(excluded_comp)

  return excluded_components

In [115]:
components

[[0, 1, 2, 3], [4, 5, 6]]

In [116]:
excluded_comps = exclude_non_terminal_nodes_from_component(TERMINAL_NODES, components)
excluded_comps

[[0, 1, 3], [5, 6]]

- Compute Pairwise connectivity considering terminal nodes

In [118]:
import math

def compute_pc_terminals(G: nx.Graph, terminal_nodes: list[int]) -> int:

  # 1. Get connected components
  components = connected_components(G)

  # 2. Exclude non-terminal nodes from the components
  excluded_components = exclude_non_terminal_nodes_from_component(terminal_nodes, components)

  # 3. Compute pairwise connectivity using comb(n, 2) = n! / (n - 2)! * 2!
  pairwise_connectivity = 0

  for comp in excluded_components:
    n = int(len(comp))
    pairwise_connectivity += math.comb(n, 2)

  return pairwise_connectivity

In [ ]:
TERMINAL_NODES = [0, 1, 2, 3, 4, 5]

pc = compute_pc_terminals(G, TERMINAL_NODES)
pc

0